In [1]:
%%capture 
!pip install langchain openai
!pip install -q -U faiss-cpu tiktoken

In [1]:
import os 
import getpass

In [3]:
os.environ["OPENAI_API_KEY"]= getpass.getpass("API KEY: ")

API KEY:  ········


Index System

Load Documents

In [4]:
from langchain.document_loaders import WebBaseLoader

yolo_nas_loader=WebBaseLoader("https://deci.ai/blog/yolo-nas-object-detection-foundation-model/").load()

decicoder_loader= WebBaseLoader("https://deci.ai/blog/decicoder-efficient-and-accurate-code-generation-llm/#:~:text=DeciCoder's%20unmatched%20throughput%20and%20low,re%20obsessed%20with%20AI%20efficiency.").load()

yolo_newsletter_loader=WebBaseLoader("https://deeplearningdaily.substack.com/p/unleashing-the-power-of-yolo-nas").load()

Chunk Documents

In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter( chunk_size=500, chunk_overlap=50, length_function=len)

yolo_nas_chunks=text_splitter.transform_documents(yolo_nas_loader)
decicoder_chunks = text_splitter.transform_documents(decicoder_loader)
yolo_newsletter_chunks = text_splitter.transform_documents(yolo_newsletter_loader)

Create Index

In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store=LocalFileStore("./cache/")

#create embedder
core_embeddings_model= OpenAIEmbeddings()

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model,
    store,
    namespace=core_embeddings_model.model
)

#store embeddings in vector store
vectorstore = FAISS.from_documents(yolo_nas_chunks,embedder)
vectorstore.add_documents(decicoder_chunks)
vectorstore.add_documents(yolo_newsletter_chunks)

#instantiate a retriever
retriever= vectorstore.as_retriever()

Retrieval System

In [9]:
from langchain_community.chat_models import ChatOpenAI

In [10]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

In [11]:
llm=ChatOpenAI(temperature=0)
handler = StdOutCallbackHandler()

In [12]:
qa_with_sources_chain= RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

Augment System

In [13]:
response = qa_with_sources_chain({"query":"What does Neural Architecture Search have to do with how Deci creates its models?"})



> Entering new RetrievalQA chain...

> Finished chain.


In [14]:
print(response['result'])

Deci uses Neural Architecture Search (NAS) technology to create its models. NAS automates the process of developing neural network architectures by intelligently searching a vast space of possible architectures. Deci's proprietary NAS technology, called AutoNAC, is used to generate innovative elements of architecture for models like DeciCoder. This allows Deci to create efficient and high-performing models that surpass other models in terms of accuracy and speed. In the case of YOLO-NAS, Deci's researchers took inspiration from existing YOLOv6 and YOLOv8 architectures and used AutoNAC to search for the best architectures within the defined search space.


In [15]:
response = qa_with_sources_chain({"query":"What is DeciCoder"})



> Entering new RetrievalQA chain...

> Finished chain.


In [16]:
print(response['result'])

DeciCoder is an open-source Large Language Model (LLM) developed by Deci. It is designed for code generation and has 1 billion parameters. DeciCoder has a 2048-context window, which allows it to consider a larger context when generating code. It offers improved throughput, accuracy, and memory usage compared to other widely-used code generation LLMs like SantaCoder. DeciCoder has been trained on a large dataset of permissively-licensed source code from various programming languages, including Python, Java, and Javascript. It can be used for commercial applications and is ready for deployment.


In [17]:
response = qa_with_sources_chain({"query":"Write a blog about Deci and how it used NAS to generate YOLO-NAS and DeciCoder"})



> Entering new RetrievalQA chain...

> Finished chain.


In [18]:
print(response['result'])

Title: Deci's Breakthrough: Unleashing the Power of NAS with YOLO-NAS and DeciCoder

Introduction:
In the ever-evolving field of artificial intelligence, Deci has emerged as a trailblazer, pushing the boundaries of accuracy and efficiency in object detection. Through their innovative use of Neural Architecture Search (NAS), Deci has introduced two groundbreaking models - YOLO-NAS and DeciCoder. These models have not only revolutionized the field but also opened up new possibilities across various industries and research domains.

Deci's Edge:
Deci's journey towards excellence began with their robust capabilities showcased by DeciCoder, which consistently outperformed models like SantaCoder. This achievement was made possible through their pioneering use of AutoNAC, a compute-efficient method that generates NAS-generated algorithms. By leveraging AutoNAC, Deci was able to create architectures that are both efficient and powerful, setting them apart from the competition.

YOLO-NAS: A Ben